In [12]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms.functional as F
from torchvision import datasets, transforms, models
import datetime
import random
import pandas as pd
import numpy as np


from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
import PIL
from PIL import Image as image


In [3]:
torch.manual_seed(42)
device = (torch.device('cuda') if torch.cuda.is_available()
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


### Dataloader

In [14]:
data_path = './retina1_trainvalid/trainvalid'

os.getcwd()
image = PIL.Image.open(f"{data_path}/0021.jpg")
width, height = image.size
print(width, height)

path, dirs, files = next(os.walk(data_path))
file_count = len(files)
file_count
f = open("./retina1_trainvalid/labels_trainvalid.txt", "r")
y_list = list()
f = f.readlines()
for i in range(len(f)):
    y_list.append(int(f[i].strip('\n')))

class SquarePad:
    def __call__(self, image):
        max_wh = max(image.size)
        p_left, p_top = [(max_wh - s) // 2 for s in image.size]
        p_right, p_bottom = [max_wh - (s+pad) for s, pad in zip(image.size, [p_left, p_top])]
        padding = (p_left, p_top, p_right, p_bottom)
        return F.pad(image, padding, 0, 'constant')

2880 2136


In [17]:
import torchvision.transforms.functional as F
#DataLoad
data_path = './retina1_trainvalid/trainvalid/'

def load_split_train_vaild(datadir, valid_size = .2):
    train_transforms = transforms.Compose([SquarePad(),
                                       transforms.Resize(224),
                                       transforms.ToTensor(), 
                                       ])
    valid_transforms = transforms.Compose([SquarePad(),
                                        transforms.Resize(224),
                                        transforms.ToTensor(),
                                        ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    valid_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=4)
    validloader = torch.utils.data.DataLoader(valid_data,
                   sampler=valid_sampler, batch_size=4)
    

    return trainloader, validloader

trainloader, validloader = load_split_train_vaild(data_path, .2)
print(trainloader.dataset[0][0].size())
plt.imshow(np.uint8(trainloader.dataset[0][0].permute(1,2,0)*255))
plt.show()

FileNotFoundError: Couldn't find any class folder in ./retina1_trainvalid/trainvalid/.